In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
import sys
import importlib

In [2]:
sys.path.append('..')

In [3]:
import modules.transforms

In [4]:
importlib.reload(modules.transforms)

<module 'modules.transforms' from '../modules/transforms.py'>

In [5]:
from modules.transforms import *

In [6]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [7]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [8]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [9]:
url="https://github.com/MinCiencia/Datos-COVID19/tree/master/output/"
request=requests.get(url)
root=url.split('github.com')[1]
prefix=url.split('/Min')[0]
folders=request.text.split(root)

#product folders
for idx in range(len(folders)):
    f=folders[idx]
    f=f.split('">')[0]
    folders[idx]=f
folders=[prefix+root+f for f in folders if 'producto' in f]

In [10]:
productos=['producto14','producto19','producto53','producto54','producto52','producto48','producto24'
    
]

In [11]:
fs=[]
for p in productos:
    fs.extend([ x for x in folders if p in x])

In [12]:
productos=pd.DataFrame(productos)

In [17]:
k='tracking'
if not cnx.dialect.has_schema(cnx, k):
    print('schema '+k+' does not exist, creating it')
    cnx.execute(sqlalchemy.schema.CreateSchema(k))
else:
    print('schema '+k+' exists, will not be created')

productos.to_sql('min_ciencias_products',con=cnx,schema=k,if_exists='replace')

schema tracking exists, will not be created


In [14]:
tables={}
for f in fs:
    files_folder=f.rsplit('/')[-1]
    files_folder_prefix='/MinCiencia/Datos-COVID19/blob/master/output/'+files_folder+'/'
    tables[files_folder]={}
    raw_path='https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/'+files_folder+'/'
    req=requests.get(f)
    files=req.text.split(files_folder_prefix)
    for idx in range(len(files)):
        files[idx]=files[idx].split('">')[0]
    files=[raw_path+f for f in files if '.csv' in f]
    for fl in files:
        try:
            print('reading file: '+fl)
            table=pd.read_csv(fl)
            table_name='_'.join(fl.rsplit('/',2)[-2:]).strip('.csv')
            tables[files_folder][table_name]=table
        except Exception as e:
            print(str(e))
            pass

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo_T.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna_T.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto53/confirmados_nacional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto53/confirmados_provinciales.c

In [15]:
tables['producto11']={}
tables['producto11']['producto4']=pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto11/bulk/producto4.csv')
tables['producto11']['producto4'].columns=[x.replace('*','x').replace('%','porcentaje_') for x in tables['producto11']['producto4'].columns]

In [16]:
ortables=copy.deepcopy(tables)

In [17]:
#normalizar tablas transpuestas y cambiar campos "fecha" (donde haya) a tipo datetime
for s in tables.keys():
    for n in tables[s].keys():
        #print(n)
        if '_T' in n[-2:]:
            print('fixing transposed table: '+n)
            tables[s][n]=transpose(tables[s][n])
        elif 'fecha' in [str(x).lower() for x in tables[s][n].columns]:
            print('converting to date in table: '+n)
            tables[s][n]=to_datetime(tables[s][n])
            

fixing transposed table: producto14_FallecidosCumulativo_T
converting to date in table: producto14_FallecidosCumulativo_std
fixing transposed table: producto19_CasosActivosPorComuna_T
converting to date in table: producto19_CasosActivosPorComuna_std
converting to date in table: producto53_confirmados_nacional
converting to date in table: producto53_confirmados_provinciale
converting to date in table: producto53_confirmados_regionale
converting to date in table: producto53_confirmados_
converting to date in table: producto54_r.nacional
converting to date in table: producto54_r.provincial
converting to date in table: producto54_r.provincial_copy
converting to date in table: producto54_r.regional
converting to date in table: producto54_r
converting to date in table: producto52_Camas_UCI_std
fixing transposed table: producto48_SOCHIMI_T
converting to date in table: producto48_SOCHIMI_std
fixing transposed table: producto24_CamasHospital_Diario_T
converting to date in table: producto24_Cama

In [18]:
#agregar variable dummy para región metropolitana
for s in tables.keys():
    for n in tables[s].keys():
        #print(n)
        if 'region' in [x.lower() for x in tables[s][n].columns]:
            print('creando variable dummy para región metropolitana en tabla: '+n)
            tables[s][n]=make_dummies(tables[s][n],fields=['region'],keep_strings=['metropolitana'],value_map={'metropolitana':{0:'Resto de Chile',1:'Región Metropolitana'}})

creando variable dummy para región metropolitana en tabla: producto14_FallecidosCumulativo
creando variable dummy para región metropolitana en tabla: producto14_FallecidosCumulativo_std
creando variable dummy para región metropolitana en tabla: producto19_CasosActivosPorComuna
creando variable dummy para región metropolitana en tabla: producto19_CasosActivosPorComuna_std
creando variable dummy para región metropolitana en tabla: producto53_confirmados_provinciale
creando variable dummy para región metropolitana en tabla: producto53_confirmados_regionale
creando variable dummy para región metropolitana en tabla: producto53_confirmados_
creando variable dummy para región metropolitana en tabla: producto54_r.provincial
creando variable dummy para región metropolitana en tabla: producto54_r.provincial_copy
creando variable dummy para región metropolitana en tabla: producto54_r.regional
creando variable dummy para región metropolitana en tabla: producto54_r
creando variable dummy para regió

In [19]:
for k in tables.keys():
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema producto14 exists, will not be created
schema producto19 exists, will not be created
schema producto53 exists, will not be created
schema producto54 exists, will not be created
schema producto52 exists, will not be created
schema producto48 exists, will not be created
schema producto24 exists, will not be created
schema producto11 exists, will not be created


In [20]:
for schema in tables.keys():
    for name in tables[schema].keys():
        try:
            df=tables[schema][name]
            name=name.replace('-','_')
            print("creating table "+name+' ,schema: '+schema)
            df.to_sql(name, schema=schema,con=cnx,if_exists='replace')
            print("saving table"+path+name+timestamp+'.csv in cache')
            df.to_csv(path+name+timestamp+'.csv',encoding='utf-8')
        except Exception as e: 
            print(str(e))
            pass

creating table producto14_FallecidosCumulativo ,schema: producto14
saving table/data/ETLcache/producto14_FallecidosCumulativo_23012021_223640.csv in cache
creating table producto14_FallecidosCumulativo_T ,schema: producto14
saving table/data/ETLcache/producto14_FallecidosCumulativo_T_23012021_223640.csv in cache
creating table producto14_FallecidosCumulativo_std ,schema: producto14
saving table/data/ETLcache/producto14_FallecidosCumulativo_std_23012021_223640.csv in cache
creating table producto19_CasosActivosPorComuna ,schema: producto19
saving table/data/ETLcache/producto19_CasosActivosPorComuna_23012021_223640.csv in cache
creating table producto19_CasosActivosPorComuna_T ,schema: producto19
saving table/data/ETLcache/producto19_CasosActivosPorComuna_T_23012021_223640.csv in cache
creating table producto19_CasosActivosPorComuna_std ,schema: producto19
saving table/data/ETLcache/producto19_CasosActivosPorComuna_std_23012021_223640.csv in cache
creating table producto53_confirmados_na

In [22]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/covid-chile.ipynb')

65280